In [36]:
# import undetected_chromedriver as uc
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Get Steam_DB genres ids and names

In [ ]:
driver = uc.Chrome()
driver.get("https://steamdb.info/tags/")
time.sleep(3)
page = driver.page_source
soup = BeautifulSoup(page,"html.parser")
labels = soup.find("div",{"id":"list"}).find_all("div",{"class":"label"})

genres = []
for label in labels:
    dic = {}
    a = label.find("a")
    id = a["href"].split("/")[-2]
    text = a.text.strip()
    dic["id"] = id
    dic["name"] = text
    genres.append(dic)

driver.close()

In [ ]:
genre_id = pd.DataFrame(genres)
genre_id.to_csv("genres_and_ids.csv")
genre_id.head()

,id,name
0,492,Indie
1,1003823,Profile Features Limited
2,19,Action
3,597,Casual
4,4182,Singleplayer


# Get all tags for game in joint.csv

In [ ]:
api_key = "6sRnydxcHkpMyGjFzXFZNZ1e6pE"
cat ={"id":[],"name":[]}
gen ={"id":[],"name":[]}
error_lst = []
result = []

In [73]:
with open("leak_ids.csv") as f:
    ids = [id.strip() for id in f.readlines()[1:]]


for app_id in ids:
    dic = {}
    page = requests.get(f"https://api.steamapis.com/market/app/{app_id}?api_key={api_key}").json()

    keys = list(page.keys())
    if "genres" in keys and "categories" in keys:

        #get all categories of the app
        categories = page["categories"]
        cat_lst = []
        for category in categories:
            cat_id = category["id"] 
            cat_name = category["description"] 

            #update category_index.csv
            if cat_id not in cat["id"]:
                cat["id"].append(cat_id)
                cat["name"].append(cat_name)
            
            cat_lst.append(str(cat_id))


        #get all genres of the app
        genres = page["genres"]
        gen_lst = []
        for genre in genres:
            gen_id = genre["id"] 
            gen_name = genre["description"] 

            #upload genre_index.csv
            if gen_id not in gen["id"]:
                gen["id"].append(gen_id)
                gen["name"].append(gen_name)
            
            gen_lst.append(str(gen_id))


        dic["id"] = app_id
        dic["category"] = ",".join(cat_lst)
        dic["genre"] = ",".join(gen_lst)
        
        result.append(dic)

    else:
        error_lst.append(app_id)



In [82]:
final = pd.DataFrame(result)
final.to_csv("leak_category_genre.csv", index = False)

In [78]:
cat_df = pd.DataFrame.from_dict(cat)
cat_df.to_csv("category_index.csv",index=False)

In [79]:
gen_df = pd.DataFrame.from_dict(gen)
gen_df.to_csv("genre_index.csv",index=False)

In [81]:
print(len(error_lst))
print(len(result))

6604
6623
